# Processamento das Entidades 

Nesse notebook, os arquivos gerados pelo BERT são processados para a geração dos arquivos finais com as entidades. O processamento consiste em eliminar algumas entidades que não ajudam muito no entendimento do texto, como: 'Presidente', 'Ministro', 'Sr', 'Sra'. O objetivo é deixar em evidência entidades que tragam mais informação para a análise dos textos processados, como por exemplo: Presidente da República, Senador Randolfe e assim por diante. 

Essa eliminação foi feita de forma manual, selecionando algumas entidades para cada uma das classes extraídas. 

Outro processamento realizado foi a unificação de termos como COVID e COVID-19. 

Por fim, as entidades das classes `COISA`, `ABSTRACCAO`, `OUTRO`, `OBRA` e `ACONTECIMENTO` foram agrupadas em uma única entidade: `OUTROS`. 

In [3]:
import pandas as pd 
import json

Palavras removidas por entidades

In [4]:
remove_words = {'PESSOA': [
    'Sr. Presidente',
    'Sr. Relator',
    'Prefeitos',
    'Presidente',
    'Senador',
    'Senadora',
    'Ministro',
    'Governador',
    'Senhor',
    'Excelência',
    'Relator',
    'V. Exa.',
    'V. Exa',
    'V. Exas',
    'Governador do',
    'Srs.',
    'Sr.',
    'Sra',
    'Sras',
    'Sra.',
    'Sras.',
    'Sr',
    'Srs',
    'Secretário de',
    'Covid-19',
    'Governador',
    'Governadores',
    'Prefeitos',
    'Sras. Senadoras',
    'Senadores', 'Senadoras', 'Sr. Senador', 'Srs. Senadores', 'Deputado', 'Secretário', 'Secretários',
    'Parlamentar', 'Parlamentares', 'Deputados', ],
    'LOCAL': ['Território', 'Brasil', 'País', 'Município', 'Municípios', 'Estado', 'Estados', 'Países', 'Covid-19'],
    "ORGANIZACAO": ['Município', 'Governo', 'Municípios', 'Estado', 'Estados', 'Países', 'País', 'Brasil', 'Covid-19', 'China'],
    "OUTROS": ['Covid-19', 'País', 'Programa', 'Medicina']

}


Mapeamento para unificar algunas alguns termos:

In [5]:
map_words = {
    'Covid': 'Covid-19',
    'Programa Nacional de Imunizações': 'PNI'
}

Classes que foram agrupadas em Outros.

In [6]:
class_to_outros = ['COISA','ABSTRACCAO','OUTRO','OBRA','ACONTECIMENTO']

O código a seguir acessa cada arquivo gerado pelo modelo de Reconhecimento de Entidades nomeadas e agrupa em duas variáveis: `all_entites` que tem todas as entidades independente do dia e `all_entities_date` que agrupa por data.

In [7]:
import os 

path_ = "../output/ner_depoentes_convidados/ner_files/"

all_files = os.listdir(path_)

all_entites_by_date = {}
all_entites = {}


for file_ in all_files:
    file_split = file_.split("_")
    date_ = file_split[1]
    name_ = ' '.join(file_split[2:]).split(".")[0]
    
    with open(path_ + file_) as datafile:
        data_ = json.load(datafile)

    final_dict = {}
    for data in data_[0]['entities']:
        text_ = data['text']
        text_ = text_.strip()

        if text_ in map_words.keys():
            text_ = map_words[text_]

        class_ = data['class']

        if class_ in class_to_outros:
            class_ = 'OUTROS'
        
        if class_ in remove_words.keys():
            words_to_remove = remove_words[class_]
        else:
            words_to_remove = []

        if text_ not in words_to_remove:



            if class_ not in final_dict.keys():
                final_dict[class_] = {text_: 1}
            else: 
                if text_ not in final_dict[class_].keys():
                    final_dict[class_][text_] = 1
                else:
                    final_dict[class_][text_] += 1

            if class_ not in all_entites.keys():
                all_entites[class_] = {text_: 1}
            else:
                if text_ not in all_entites[class_].keys():
                    all_entites[class_][text_] = 1
                else:
                    all_entites[class_][text_] += 1

            if date_ not in all_entites_by_date.keys():
                all_entites_by_date[date_] = {}

            if class_ not in all_entites_by_date[date_].keys():
                all_entites_by_date[date_][class_] = {text_: 1}
            else: 
                if text_ not in all_entites_by_date[date_][class_].keys():
                    all_entites_by_date[date_][class_][text_] = 1
                else:
                    all_entites_by_date[date_][class_][text_] += 1


    for class_ in final_dict.keys():
        final_dict[class_] = dict(sorted(final_dict[class_].items(), key=lambda item: item[1], reverse=True))

    for class_ in all_entites.keys():
        all_entites[class_] = dict(sorted(all_entites[class_].items(), key=lambda item: item[1], reverse=True))

    for date_ in all_entites_by_date.keys():
        for class_ in all_entites_by_date[date_].keys():
            all_entites_by_date[date_][class_] = dict(sorted(all_entites_by_date[date_][class_].items(), key=lambda item: item[1], reverse=True))
        

Os dicionários criados são salvos em dois arquivos na pasta `output`: 

* `all_entites`: foi salvo em `todas_entidades_nomeadas.json`.
* `all_entites_by_date`: foi salvo em `todas_entidades_nomeadas_por_data.json`

In [203]:
with open("../output/todas_entidades_nomeadas.json", 'w') as outfile:
    json.dump(all_entites, outfile)

In [204]:
with open("../output/todas_entidades_nomeadas_por_data.json", 'w') as outfile:
    json.dump(all_entites_by_date, outfile)